In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import sklearn


# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
#!pip install tensorflow

In [3]:
#load data from data files and place into panda dataframes
custOrders = pd.read_csv('data/customer_orders.csv', low_memory=False)
temp2 = pd.read_csv('data/item_ingredients.csv', low_memory=False)

#temp2.head(10)


In [4]:
#turn off panda max column display limit
print(pd.options.display.max_columns)
pd.options.display.max_columns
mns = None

def dropCol(dataFrame, colName):
    return dataFrame.drop(colName, axis=1)
#temp = temp.drop('gender', axis=1)


20


In [5]:
#convert the boolean values into integers in the ingredients dataframe
#also print column names of dataframe
for col in temp2.columns:
    print(col)
    temp2[col] = temp2[col].astype(int)


item_id
has_ketchup
has_mustard
has_mayonnaise
has_lettuce
has_pickles
has_onions
has_tomatoes
has_cheese
has_beef
has_bun
has_salt
has_potatoes
has_coffee
has_milk
has_sugar
has_soda
has_wheat
has_ice_cream
has_bacon


In [6]:
#print the binary version of the item-ingredient dataframe
temp2.head()
temp2.shape

(21, 20)

In [7]:
#change type to datetime
custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])
custOrders.head()

,customer_id,opt_in,birthday,gender,order_id,weather,order_timestamp,item_id,item_name,item_description,item_image,price,taste_profile,item_type,customization_id,customization
0,170,True,1997-11-14,male,390,snowy,2019-09-01 21:27:18,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,27.0,no onions
1,152,True,1999-11-14,male,1620,cloudy,2017-05-22 14:08:07,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
2,186,True,1993-11-27,male,782,sunny,2019-01-16 04:50:55,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
3,150,True,1994-03-07,female,1571,rainy,2019-06-19 00:30:29,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,25.0,no lettuce
4,40,True,2003-07-04,female,240,sunny,2020-07-08 23:50:39,640404963,Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.29,savory,burger,9.0,no mustard


In [8]:
custOrders.head()

,customer_id,opt_in,birthday,gender,order_id,weather,order_timestamp,item_id,item_name,item_description,item_image,price,taste_profile,item_type,customization_id,customization
0,170,True,1997-11-14,male,390,snowy,2019-09-01 21:27:18,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,27.0,no onions
1,152,True,1999-11-14,male,1620,cloudy,2017-05-22 14:08:07,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
2,186,True,1993-11-27,male,782,sunny,2019-01-16 04:50:55,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
3,150,True,1994-03-07,female,1571,rainy,2019-06-19 00:30:29,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,25.0,no lettuce
4,40,True,2003-07-04,female,240,sunny,2020-07-08 23:50:39,640404963,Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.29,savory,burger,9.0,no mustard


In [9]:
#TODO - drop columns that are optional and come in with opt-ins
#opt_in, birthday, gender(anything else?)
#customization, customization_id
#item_image, 

#? unsure ----> item_name, item_description
#Think need more item related features, 

#function converts a cyclical data point(like time) into a 2 point representation on a circle 
def cycleEmbed(maximum, reading):
    maxi=maximum

    sin = np.sin(2*np.pi*reading/maxi)
    cos = np.cos(2*np.pi*reading/maxi)
    return (sin, cos)



In [10]:
#function takes a column name, dataframe, and then converts the column in the dataframe into a integers, 
#where each integer represents a unique value from that column
def enumerateCol(colName, dataFrame):
    count=0
    diction={}

    for i, row in dataFrame.iterrows():
        currentVal=row.loc[colName]
        if currentVal in diction:
            dataFrame.at[i, colName] = diction[currentVal]
        else:
            diction[currentVal]=count
            count+=1
            dataFrame.at[i, colName] = diction[currentVal]
            
        



In [11]:
#dropCol(temp, "item_image")
custOrders.head(15)

,customer_id,opt_in,birthday,gender,order_id,weather,order_timestamp,item_id,item_name,item_description,item_image,price,taste_profile,item_type,customization_id,customization
0,170,True,1997-11-14,male,390,snowy,2019-09-01 21:27:18,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,27.0,no onions
1,152,True,1999-11-14,male,1620,cloudy,2017-05-22 14:08:07,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
2,186,True,1993-11-27,male,782,sunny,2019-01-16 04:50:55,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
3,150,True,1994-03-07,female,1571,rainy,2019-06-19 00:30:29,640405085,Double Hamburger,Our top-quality hamburgers are made from hormo...,NaN,2.99,savory,burger,25.0,no lettuce
4,40,True,2003-07-04,female,240,sunny,2020-07-08 23:50:39,640404963,Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.29,savory,burger,9.0,no mustard
5,46,True,1990-11-18,female,1841,cloudy,2019-12-02 14:50:30,640405172,Double Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.79,savory,burger,47.0,no lettuce
6,123,True,2002-02-15,male,1159,snowy,2020-08-20 07:43:06,640405025,Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.79,savory,burger,41.0,no onions
7,38,True,1990-01-21,male,1333,snowy,2018-03-26 00:15:40,640405112,Double Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,3.29,savory,burger,34.0,no onions
8,22,True,2002-05-10,female,1132,snowy,2016-06-09 22:10:25,640405058,Veggie Burger,Our veggie burger is fresh and delicious.,NaN,2.49,healthy,burger,20.0,no onions
9,92,True,1992-01-24,female,334,snowy,2020-01-06 07:13:25,640405025,Bacon Cheeseburger,Our top-quality hamburgers are made from hormo...,NaN,2.79,savory,burger,40.0,no pickles


In [12]:
# Get a Dictionary containing the pairs of column names & data type objects.

print('Data type of each column of Dataframe :')
print(dict(custOrders.dtypes))


Data type of each column of Dataframe :
{'customer_id': dtype('int64'), 'opt_in': dtype('bool'), 'birthday': dtype('O'), 'gender': dtype('O'), 'order_id': dtype('int64'), 'weather': dtype('O'), 'order_timestamp': dtype('<M8[ns]'), 'item_id': dtype('int64'), 'item_name': dtype('O'), 'item_description': dtype('O'), 'item_image': dtype('O'), 'price': dtype('float64'), 'taste_profile': dtype('O'), 'item_type': dtype('O'), 'customization_id': dtype('float64'), 'customization': dtype('O')}


In [13]:
#split timestamp in "order_timestamp" into a number of individual columns for each part of the timestamp

custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])

custOrders["order_date"]=custOrders["order_timestamp"].dt.date
custOrders["order_year"]=custOrders["order_timestamp"].dt.year
custOrders["order_month"]=custOrders["order_timestamp"].dt.month
custOrders["order_day"]=custOrders["order_timestamp"].dt.day
custOrders["order_hour"]=custOrders["order_timestamp"].dt.hour
custOrders["order_minute"]=custOrders["order_timestamp"].dt.minute
custOrders["order_second"]=custOrders["order_timestamp"].dt.second

#create a column for the weekday(mon,tues, weds, ...etc)
#must convert to weekday below
custOrders["order_weekday"]=custOrders["order_day"]

for i, row in custOrders.iterrows():
    custOrders.at[i, "order_weekday"] = datetime.datetime(row["order_year"], row["order_month"],row["order_day"],0,0,0).weekday()


In [14]:
#drop columns which are optional(not all customers will use it), clearly not useful, or redundant

custOrders = custOrders.drop("item_image", axis=1, errors='ignore')
custOrders = custOrders.drop("order_timestamp", axis=1, errors='ignore') #information moved into other columns added above
custOrders = custOrders.drop("customization_id", axis=1, errors='ignore') #customization is no in every order
custOrders = custOrders.drop("customization", axis=1, errors='ignore')
custOrders = custOrders.drop("opt_in", axis=1, errors='ignore')
custOrders = custOrders.drop("item_description", axis=1, errors='ignore')

custOrders = custOrders.drop("birthday", axis=1, errors='ignore') #check if part of opt in
custOrders = custOrders.drop("gender", axis=1, errors='ignore') #check if part of opt in
#custOrders = custOrders.drop("item_name", axis=1) #reduntance since we have item id? we need 

custOrders = custOrders.drop("order_date", axis=1, errors='ignore') #check if part of opt in


custOrders.head(10)


,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,1,21,27,18,6
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,22,14,8,7,0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,16,4,50,55,2
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,19,0,30,29,2
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,8,23,50,39,2
5,46,1841,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,12,2,14,50,30,0
6,123,1159,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,8,20,7,43,6,3
7,38,1333,snowy,640405112,Double Cheeseburger,3.29,savory,burger,2018,3,26,0,15,40,0
8,22,1132,snowy,640405058,Veggie Burger,2.49,healthy,burger,2016,6,9,22,10,25,3
9,92,334,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,1,6,7,13,25,0


In [15]:
#custOrders will represet the data before it has been enumerated and is only numbers
custOrders2=custOrders.copy()
custOrders2.head()

,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,1,21,27,18,6
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,22,14,8,7,0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,16,4,50,55,2
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,19,0,30,29,2
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,8,23,50,39,2


In [16]:
#embed time values to cycles on a circle
def timeEmbed(df, colName, maxi):
    name1=colName+"1"
    name2=colName+"2"
    print(name1)
    print(name2)
    df[name1]=None
    df[name2]=None
    for i, row in df.iterrows():
        temp=cycleEmbed(maxi, row[colName])
        df.at[i, name1]=temp[0]
        df.at[i, name2]=temp[1]

#convert every time based column into a point on a circle with timeEmbed()    
timeEmbed(custOrders2, "order_month", 12)
timeEmbed(custOrders2, "order_day", 31)
timeEmbed(custOrders2, "order_hour", 24)
timeEmbed(custOrders2, "order_minute", 60)
timeEmbed(custOrders2, "order_second", 60)
timeEmbed(custOrders2, "order_weekday", 7)



order_month1
order_month2
order_day1
order_day2
order_hour1
order_hour2
order_minute1
order_minute2
order_second1
order_second2
order_weekday1
order_weekday2


In [17]:
#print all number version of the data

custOrders2.head(10)

,customer_id,order_id,weather,item_id,item_name,price,taste_profile,item_type,order_year,order_month,...,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,170,390,snowy,640405085,Double Hamburger,2.99,savory,burger,2019,9,...,0.201299,0.97953,-0.707107,0.707107,0.309017,-0.951057,0.951057,-0.309017,-0.781831,0.62349
1,152,1620,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2017,5,...,-0.968077,-0.250653,-0.5,-0.866025,0.743145,0.669131,0.669131,0.743145,0.0,1.0
2,186,782,sunny,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,1,...,-0.101168,-0.994869,0.866025,0.5,-0.866025,0.5,-0.5,0.866025,0.974928,-0.222521
3,150,1571,rainy,640405085,Double Hamburger,2.99,savory,burger,2019,6,...,-0.651372,-0.758758,0.0,1.0,0.0,-1.0,0.104528,-0.994522,0.974928,-0.222521
4,40,240,sunny,640404963,Cheeseburger,2.29,savory,burger,2020,7,...,0.998717,-0.050649,-0.258819,0.965926,-0.866025,0.5,-0.809017,-0.587785,0.974928,-0.222521
5,46,1841,cloudy,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2019,12,...,0.394356,0.918958,-0.5,-0.866025,-0.866025,0.5,0.0,-1.0,0.0,1.0
6,123,1159,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,8,...,-0.790776,-0.612106,0.965926,-0.258819,-0.978148,-0.207912,0.587785,0.809017,0.433884,-0.900969
7,38,1333,snowy,640405112,Double Cheeseburger,3.29,savory,burger,2018,3,...,-0.848644,0.528964,0.0,1.0,1.0,0.0,-0.866025,-0.5,0.0,1.0
8,22,1132,snowy,640405058,Veggie Burger,2.49,healthy,burger,2016,6,...,0.968077,-0.250653,-0.5,0.866025,0.866025,0.5,0.5,-0.866025,0.433884,-0.900969
9,92,334,snowy,640405025,Bacon Cheeseburger,2.79,savory,burger,2020,1,...,0.937752,0.347305,0.965926,-0.258819,0.978148,0.207912,0.5,-0.866025,0.0,1.0


In [18]:
#begin splitting custOrders2 into training/validation sets, including moving supervision target(itemID) out

masterDS=custOrders2.copy()
masterDS = masterDS.drop("order_month", axis=1, errors='ignore')
masterDS = masterDS.drop("order_day", axis=1, errors='ignore')
masterDS = masterDS.drop("order_hour", axis=1, errors='ignore')
masterDS = masterDS.drop("order_year", axis=1, errors='ignore')
masterDS = masterDS.drop("order_minute", axis=1, errors='ignore')
masterDS = masterDS.drop("order_second", axis=1, errors='ignore')
masterDS = masterDS.drop("order_weekday", axis=1, errors='ignore')

masterDS = masterDS.drop("item_id", axis=1, errors='ignore')

masterDS = masterDS.drop("item_image", axis=1, errors='ignore')
masterDS = masterDS.drop("birthday", axis=1, errors='ignore')



In [19]:
#Enumerate columns that are categorical
enumerateCol("weather", masterDS)
enumerateCol("item_name", masterDS)
enumerateCol("taste_profile", masterDS)
enumerateCol("item_type", masterDS)


In [20]:
masterDS.head(10)
#? need to normalize?
# DO I NEED TO DO the FLOAT 64 CAST? 
#cast all columns to float64
for col in masterDS.columns:
    masterDS[col] = masterDS[col].astype(np.float64)

In [21]:
target_column = ['item_name'] 
predictors = list(set(list(masterDS.columns))-set(target_column))
print(predictors)

#next line regularizes the all columns outside of target_column
# I definitly dont want it for the time data points, but do other columns need it?????

#masterDS[predictors] = masterDS[predictors]/masterDS[predictors].max()

['weather', 'item_type', 'order_minute1', 'order_second1', 'order_day2', 'order_hour1', 'order_minute2', 'order_hour2', 'order_id', 'customer_id', 'order_second2', 'order_weekday1', 'taste_profile', 'order_weekday2', 'price', 'order_month2', 'order_month1', 'order_day1']


In [22]:
X = masterDS[predictors].values
y = masterDS[target_column].values

In [23]:
#split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(1491, 18)
(640, 18)


In [24]:
# Define model
model = Sequential()
model.add(Dense(500, input_dim=X_train.shape[1], activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
#model.summary() #Print model Summary


In [25]:
#tf.keras.backend.set_floatx('float64')

model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
47/47 [==============================] - 1s 2ms/step - loss: 122.7511 - mean_squared_error: 122.7511
Epoch 2/100
47/47 [==============================] - 0s 2ms/step - loss: 47.5895 - mean_squared_error: 47.5895
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 45.2562 - mean_squared_error: 45.2562
Epoch 4/100
47/47 [==============================] - 0s 3ms/step - loss: 42.6496 - mean_squared_error: 42.6496
Epoch 5/100
47/47 [==============================] - 0s 4ms/step - loss: 47.0703 - mean_squared_error: 47.0703
Epoch 6/100
47/47 [==============================] - 0s 3ms/step - loss: 47.0977 - mean_squared_error: 47.0977
Epoch 7/100
47/47 [==============================] - 0s 3ms/step - loss: 38.6818 - mean_squared_error: 38.6818
Epoch 8/100
47/47 [==============================] - 0s 3ms/step - loss: 27.7721 - mean_squared_error: 27.7721
Epoch 9/100
47/47 [==============================] - 0s 4ms/step - loss: 33.8449 - mean_squared_error: 33.8449

47/47 [==============================] - 0s 2ms/step - loss: 10.0268 - mean_squared_error: 10.0268
Epoch 75/100
47/47 [==============================] - 0s 2ms/step - loss: 10.7358 - mean_squared_error: 10.7358
Epoch 76/100
47/47 [==============================] - 0s 2ms/step - loss: 11.1565 - mean_squared_error: 11.1565
Epoch 77/100
47/47 [==============================] - 0s 2ms/step - loss: 11.6068 - mean_squared_error: 11.6068
Epoch 78/100
47/47 [==============================] - 0s 4ms/step - loss: 10.1326 - mean_squared_error: 10.1326
Epoch 79/100
47/47 [==============================] - 0s 3ms/step - loss: 11.1428 - mean_squared_error: 11.1428
Epoch 80/100
47/47 [==============================] - 0s 2ms/step - loss: 12.1191 - mean_squared_error: 12.1191
Epoch 81/100
47/47 [==============================] - 0s 3ms/step - loss: 10.8479 - mean_squared_error: 10.8479
Epoch 82/100
47/47 [==============================] - 0s 4ms/step - loss: 11.1206 - mean_squared_error: 11.1206
Epoch

In [26]:
# Get a Dictionary containing the pairs of column names & data type objects.
dataTypeDict = dict(masterDS.dtypes)
print('Data type of each column of Dataframe :')
print(dataTypeDict)

Data type of each column of Dataframe :
{'customer_id': dtype('float64'), 'order_id': dtype('float64'), 'weather': dtype('float64'), 'item_name': dtype('float64'), 'price': dtype('float64'), 'taste_profile': dtype('float64'), 'item_type': dtype('float64'), 'order_month1': dtype('float64'), 'order_month2': dtype('float64'), 'order_day1': dtype('float64'), 'order_day2': dtype('float64'), 'order_hour1': dtype('float64'), 'order_hour2': dtype('float64'), 'order_minute1': dtype('float64'), 'order_minute2': dtype('float64'), 'order_second1': dtype('float64'), 'order_second2': dtype('float64'), 'order_weekday1': dtype('float64'), 'order_weekday2': dtype('float64')}


In [27]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

#model.predict([40.0,139.0, 0.0, 4.99, 0.00, ])

print(pred_train)

3.053385390188841
[[ 2.3172674]
 [13.520831 ]
 [13.343458 ]
 ...
 [13.740855 ]
 [ 1.9918318]
 [11.886155 ]]
